This notebook is to load predicted mask for hard-disk and calculate Segmentation evaluation metics.
predicted mask is the output of ScaleNASv2 Test and save predicted mask.ipynb

cp -v /mnt/fast-data/mjc/AutoRECIST/Codes/ScaleNAS/ScaleNASv1/tools/utils_test.py .

gt box are loaded from /cache/*gt_roidb.pkl
gt segmentation are loaded from Hao's Raw files

predictions are all_boxes and all_segms which both are loaded from mask_1988 png files.

In [1]:
# coding: utf-8

# # This file is for segmetation metrics evaluation in 3D
# Edited by Jingchen around 06/20/2021
# This file is after ScaleNAS test which save predition into png images.
# This file load png images as predicted contours in 2D
# load cache pkl as gold-standard contours in 2D
# The stack 2D based on dicom-header to get 3D
# Evaluate 3D metics of dice, IoU, over-segmetation, and under-segmetation
# 
%reload_ext autoreload
%autoreload 2

import os
import logging
import numpy as np
import _init_paths

from config import cfg
from config import update_config
from utils_test import *

import cv2
from PIL import Image
import torch.nn.functional as F
from utils.utils import get_confusion_matrix
def convert_name(name):
    new = name.replace('/','_')
    return new

def get_palette(n):
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab:
            palette[j * 3 + 0] |= (((lab >> 0) & 1) << (7 - i))
            palette[j * 3 + 1] |= (((lab >> 1) & 1) << (7 - i))
            palette[j * 3 + 2] |= (((lab >> 2) & 1) << (7 - i))
            i += 1
            lab >>= 3
    return palette
    
# def save_pred(preds, sv_path, name):

#     preds = preds.cpu().numpy().copy()
#     preds = np.asarray(np.argmax(preds, axis=1), dtype=np.uint8)
#     for i in range(preds.shape[0]):
#         cv2.imwrite(os.path.join(sv_path, convert_name(name[i])) , preds[i])

        
def save_pred( preds, sv_path, name):
    palette = get_palette(256)
    preds = preds.cpu().numpy().copy()
    preds = np.asarray(np.argmax(preds, axis=1), dtype=np.uint8)
    for i in range(preds.shape[0]):
        pred = preds[i]
        save_img = Image.fromarray(pred)
        save_img.putpalette(palette)
        save_img.save(os.path.join(sv_path, convert_name(name[i]) ))
        

def testval_lesion(config, test_dataset, testloader, model,
            sv_dir='', sv_pred=True, device = None):
    model.eval()
    confusion_matrix = np.zeros(
        (config.DATASET.NUM_CLASSES, config.DATASET.NUM_CLASSES))
    with torch.no_grad():
        for index, batch in enumerate(testloader):
            image, label, _, batchname = batch
            size = label.size()
            if device is None:
                image = image.cuda()
                label = label.long().cuda()
            else:
                image = image.to(device)
                label = label.long().to(device)

            pred = model(image)
            if pred.size()[-2] != size[-2] or pred.size()[-1] != size[-1]:
                pred = F.upsample(pred, (size[-2], size[-1]),
                                  mode='bilinear')

            confusion_matrix += get_confusion_matrix(
                label,
                pred,
                size,
                config.DATASET.NUM_CLASSES,
                config.TRAIN.IGNORE_LABEL)

            if sv_pred:
                sv_path = os.path.join(sv_dir, 'test_val_results')
                if not os.path.exists(sv_path):
                    os.mkdir(sv_path)
                save_pred(pred, sv_path, batchname)

            if index % 100 == 0:
                logging.info('processing: %d images' % index)
                pos = confusion_matrix.sum(1)
                res = confusion_matrix.sum(0)
                tp = np.diag(confusion_matrix)
                IoU_array = (tp / np.maximum(1.0, pos + res - tp))
                mean_IoU = IoU_array.mean()
                logging.info('mIoU: %.4f' % (mean_IoU))

    pos = confusion_matrix.sum(1)
    res = confusion_matrix.sum(0)
    tp = np.diag(confusion_matrix)
    pixel_acc = tp.sum() / pos.sum()
    mean_acc = (tp / np.maximum(1.0, pos)).mean()
    IoU_array = (tp / np.maximum(1.0, pos + res - tp))
    mean_IoU = IoU_array.mean()

    return mean_IoU, IoU_array, pixel_acc, mean_acc

In [3]:

site_list = [2,4,6,8,9,10,11,14,17] 

cache_path = './cache/'
name = 'inference'
# name = 'lesion_train'

cache_filepath = os.path.join(cache_path, name+'_gt_roidb.pkl')
print('Loading cached gt_roidb from %s', cache_filepath)
with open(cache_filepath, 'rb') as fp:
    cached_roidb = pickle.load(fp)
    
roidb = cached_roidb


from utils_test import __get_annotation__

sv_dir = mask_name = 'mask_1988'
sv_path = os.path.join(sv_dir, 'test_val_results')

all_boxes = [ [ np.zeros((0,5),dtype="float32") for _ in range(len(roidb)) ] for _ in range( cfg.DATASET.NUM_CLASSES) ]
all_segms = [ [ [] for _ in range(len(roidb)) ] for _ in range( cfg.DATASET.NUM_CLASSES) ]

for i in range(len(roidb)):
    if i%500==0:
        print('='*40,i)

    one = roidb[i]
    onename = one['image']
    if not os.path.exists( os.path.join( sv_path, convert_name(onename) ) ):
        print(os.path.join(sv_path, convert_name(onename) ) , 'not exists!')
    pred_im = Image.open(os.path.join( sv_path, convert_name(onename) ))
    pred = np.array(pred_im)
    for j in range(cfg.DATASET.NUM_CLASSES):
        mask = np.asarray( pred==j , dtype=np.uint8)
        if np.sum(mask > 0) <= 3 :
            continue
        segmentation, bbox, area = __get_annotation__(mask , xywh = False , bbox_score=True)
        if segmentation and bbox:
            all_segms[j][i] = segmentation
            all_boxes[j][i] = bbox




Loading cached gt_roidb from %s ./cache/inference_gt_roidb.pkl
======================================== 0


In [4]:
D_CT = {}
for i , aroidb in enumerate(roidb):
    dicom_path , png_name = os.path.split(aroidb['image'])
    slice_no , _= os.path.splitext(png_name)
    slice_no = int(slice_no)
    if slice_no != aroidb['slice_no']:
        print('following slice numbers are not consistence.')
        print(dicom_path,slice_no,aroidb['slice_no'])

    segmentations = {}
    bboxes = {}
    for j in site_list:
        segmentations[j] = all_segms[j][i]
        bboxes[j] = all_boxes[j][i]

    if dicom_path not in D_CT:
        D_CT[dicom_path] = {}
        D_CT[dicom_path][slice_no] = [aroidb , bboxes , segmentations]
    else:
        D_CT[dicom_path][slice_no] = [aroidb , bboxes , segmentations]

In [7]:
SHOW_LABEL = True
SHOW_BOX = False
SHOW_MASK = True
SHOW_UNION_MASK= False
SHOW_MASK_LABEL = True
SHOW_GT_MASK = False
from utils_metrics_3d import *


def get_proper_CT_windowing(segmentations):

    if 8 in segmentations and segmentations[8] :
        return -160 , 240
    if 7 in segmentations and segmentations[7] :
        return -160 , 240    
    if 9 in segmentations and segmentations[9] :
        return -1250 , 250
    return None

keys = list(D_CT.keys())

for k in keys:
    
    oneCT = remove_single_slice_segms(D_CT[k])
    savepath = '/mnt/fast-disk1/demo/%s/CTs_%s_allsites/%s'%(mask_name,name,convert_name_compact(k))
    if not os.path.exists(savepath):
        os.makedirs(savepath)
    
    for s in oneCT:
        aroidb , bboxes , segmentations = oneCT[s]

        image_path = os.path.join(aroidb['image'])
    
        CT_windowing = get_proper_CT_windowing(segmentations)
        if CT_windowing:
            HU1, HU2 = CT_windowing
        else:
            [HU1, HU2 ] = aroidb['windows']
            

        image = load_image(image_path, HU1, HU2)
        height,width = image.shape
    #     plt.imshow(image)
        image = np.dstack((image,image,image))
        
        if SHOW_MASK:
            for j in site_list:
                contours = segmentations[j]
                colors = compute_colors_for_labels(j)
                label = ix2labelname(j)
                for c in contours:
                    c = np.reshape(c,(-1,2))
                    if c.shape[0]:
                        image = cv2.drawContours(image, [np.int64( c  )], -1, colors, 1)
                        if SHOW_MASK_LABEL:
                            x,y,_,_ =ploy2boxes(c)
                            template = "{}"
                            if len(label)>=9:
                                label=label[:3]+label[-3:]
                
                            s = template.format(label)
                            cv2.putText(
                                image, s, (x, y), cv2.FONT_HERSHEY_SIMPLEX, .5, colors, 1
                            )  
        

        cv2.imwrite( os.path.join(savepath, convert_name_compact( aroidb['image'] ) ) , image )

/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0006/D2018_12_11/E8314/CT/S0002_5268
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0028/D2019_07_12/E6851/CT/S0006_8606
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0195/D2019_04_08/E6219/CT/S0003_8373


In [ ]:
cd  /Users/jingchen/OneDrive/OneDrive - cumc.columbia.edu/AutoRECIST/ImageResults/ScaleNAS_Q5/mask_1514/
rsync -avzh --progress jm4669@radio-gpu.cpmc.columbia.edu:/mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5/mask_1514/Images_Cougar302_80pts_test_allsites . 
    



In [ ]:
case = 1668
pname = 'BAILN'
for aroidb in roidb:
    if pname in aroidb['image']:
        print(aroidb['id'])

convert_name( aroidb['image'] )

aroidb